In [148]:
#Self-hyperparam selection: https://link.springer.com/article/10.1007/s11063-024-11578-0
#Self-pruning: https://github.com/skarifahmed/seMLP/blob/main/src/Prune.py

In [169]:
from utils import Utils
from color import color 
import pandas as pd
import numpy as np
import os
# libraries
import joblib

# scale features
from sklearn import preprocessing
from sklearn import impute
# classifier
from sklearn.ensemble import ExtraTreesClassifier
# scoring metrics
from sklearn.metrics import confusion_matrix, matthews_corrcoef

# custom scripts
import sys
sys.path.insert(0, "%s" % "CV/")
from sklearn.model_selection import train_test_split, GridSearchCV, GroupShuffleSplit, StratifiedShuffleSplit, cross_validate, StratifiedKFold
from sklearn.metrics import roc_curve, auc, recall_score, accuracy_score, precision_score, confusion_matrix, make_scorer, matthews_corrcoef, jaccard_score

In [150]:
site_path = "/Users/sanjanayasna/csc334/MLP_MAHOMES/data/sites_calculated_features.txt"

In [151]:
#read in feature set:
sites = pd.read_csv(site_path)
sites = sites.set_index('SITE_ID',drop=True)

# The following labels need to be changed after looking over literature (see Feehan, Franklin, Slusky 2021)
change_site_labels = ["5zb8_0", "6aci_0", "6oq7_0", "6pjv_1", "6q55_0",
                      "6q55_2", "6rmg_0", "6rtg_0", "6rw0_0", "6v77_0"]

# The following sites are removed due to unkopwn correct labels (see Feehan, Franklin, Slusky 2021)
sites.loc[sites.index.isin(change_site_labels), 'Catalytic']=True
remove_sites = ["6mf0_1", "6okh_0", "6qwo_0", "6r9n_0"]
sites=sites.loc[~sites.index.isin(remove_sites)]

#print shape of dataset
print(color.BOLD + "All features:" + color.END)
print("sites: %s \tcolumns: %s"%(sites.shape[0], sites.shape[1]))
sizes = sites.groupby(["Set", "Catalytic"]).size()
print(sizes)

All features:
sites: 3981 	columns: 485
Set   Catalytic
data  False        2636
      True          829
test  False         345
      True          171
dtype: int64


In [152]:
#save_models toggel
save_models = False
#pkl output path
pkl_out = r'/Users/sanjanayasna/csc334/MLP_MAHOMES/pkl'

In [153]:
sites.head()

,Catalytic,MetalCodes,MetalAtoms,fa_atr_Sum_3.5,fa_rep_Sum_3.5,fa_sol_Sum_3.5,fa_intra_atr_xover4_Sum_3.5,fa_intra_rep_xover4_Sum_3.5,fa_intra_sol_xover4_Sum_3.5,lk_ball_Sum_3.5,...,geom_cn8,geom_cn9,geom_Filled,geom_PartFilled,geom_AvgN,geom_AvgO,geom_AvgS,geom_AvgOther,SC_vol_perc,Set
SITE_ID,,,,,,,,,,,,,,,,,,,,,
6s9z_0,True,1,1,-33.20757,20.22373,26.34441,-1.88617,0.46054,2.14096,14.05052,...,0.0,0.0,0.0,1.0,3.0,0.0,0.0,0.0,0.910384,test
6g5l_0,True,1,1,-27.04899,39.17134,22.76555,-1.71942,0.45999,2.05517,12.94894,...,0.0,0.0,0.0,1.0,3.0,0.0,0.0,0.0,0.862189,test
6hwz_0,True,1,1,-27.30433,35.04867,23.45195,-1.62146,0.35902,1.91231,13.06378,...,0.0,0.0,0.0,1.0,3.0,0.0,0.0,0.0,0.991431,test
6qww_0,True,1,1,-25.36664,12.54178,27.17902,-1.14349,0.22087,1.68091,11.47631,...,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.864546,test
6qww_1,False,1,1,-30.53159,8.99318,27.77842,-1.00782,0.39657,1.04229,13.23736,...,0.0,0.0,1.0,0.0,0.0,4.0,0.0,1.0,0.990893,test


In [154]:
#Get scaled features
data_scaled, Tsites_scaled = Utils.get_scaled_features(sites =sites, pkl_out=pkl_out, save_models=save_models)
#Print stats
print(color.BOLD + "All scaled data-set features:" + color.END)
print("sites: %s \tcolumns: %s"%(data_scaled.shape[0], data_scaled.shape[1]))
print(data_scaled.groupby(["Catalytic"]).size())

print(color.BOLD + "\nAll scaled T-metal-site features:" + color.END)
print("sites: %s \tcolumns: %s"%(Tsites_scaled.shape[0], Tsites_scaled.shape[1]))
print(Tsites_scaled.groupby(["Catalytic"]).size())

All scaled data-set features:
sites: 3465 	columns: 484
Catalytic
False    2636
True      829
dtype: int64

All scaled T-metal-site features:
sites: 516 	columns: 484
Catalytic
False    345
True     171
dtype: int64


In [155]:
dir = "/Users/sanjanayasna/csc334/MLP_MAHOMES/data/"
#save the scaled data
data_scaled.to_csv(os.path.join(dir, "data_scaled.csv"))
Tsites_scaled.to_csv(os.path.join(dir, "Tsites_scaled.csv"))

In [156]:
#set feature set type
MAHOMES_feature_set = "AllMeanSph"

In [157]:
#Well sampled training data
#X is train
#y is target for train
X, y = Utils.get_training_data(MAHOMES_feature_set, random_seed = 1, data_scaled= data_scaled)
 ## prepare test-set
testX = Tsites_scaled.copy()
testY = testX['Catalytic']; del testX['Catalytic']
testX = Utils.feature_subset(testX, MAHOMES_feature_set, noBSA=True)

## get multiple predictions for test-set w/ diff random seeds
test_site_preds = {'actual': pd.Series(testY, index=testX.index)}

#Overview:
# X: training data
# y: target for training data
# testX: test data
# testY: target for test data

In [158]:
#Train input feed params
init_features = len(X.columns)
twice = init_features * 2
print(init_features)

181


In [159]:
#Prelim mlp
#Possible avenue for bias and weight matrix initialization:
## Initialize weights using Xavier uniform initialization
# init.xavier_uniform_(linear_layer.weight)
 
# ## Initialize bias to zero
# init.zeros_(linear_layer.bias)
#---------------------------------
import torch
from torch import nn
from torch.utils.data import DataLoader
#Previous experiment
class MLP(nn.Module):  # nn.Module is the base class for all models in PyTorch
    def __init__(self):
        super().__init__()
        self.layers = nn.Sequential(
            
            nn.Linear(init_features, 181),
            nn.ReLU(),
            nn.Linear(181, 90),
            nn.ReLU(),
            nn.Linear(90, 1),
            nn.Sigmoid()
            #try to make output binary (0 or 1)
        )
    def forward(self, x):
     #   x =  self.layers(x)
        return self.layers(x)

#Deeper model
# class MLP(nn.Module):
#     def __init__(self):
#         super().__init__()
#         #wide input
#         self.layers = nn.Sequential(
#             nn.Linear(init_features, init_features),
#             nn.ReLU(),
#             nn.Linear( init_features, 60),
#             nn.ReLU(),
#             nn.Linear(60, 60),
#             nn.ReLU(),
#             nn.Linear(60, 1),
#             nn.Sigmoid()
#         )
#         # self.layer1 = nn.Linear((init_features * 2), init_features)
#         # self.act1 = nn.ReLU()
#         # self.layer2 = nn.Linear(init_features, 60)
#         # self.act2 = nn.ReLU()
#         # self.layer3 = nn.Linear(60, 60)
#         # self.act3 = nn.ReLU()
#         # self.output = nn.Linear(60, 1)
#         # self.sigmoid = nn.Sigmoid()
 
#     def forward(self, x):
#         # x = self.act1(self.layer1(x))
#         # x = self.act2(self.layer2(x))
#         # x = self.act3(self.layer3(x))
#         # x = self.sigmoid(self.output(x))
#         x = self.layers(x)
#         return self.layers(x)

In [136]:
#Loads to torch tensors
class dataLoader:
    #Use ONLY train data 
    def __init__(self, X, y):
        #converts x and y to numpy arr so they can be torch tensor
        if not torch.is_tensor(X) and not torch.is_tensor(y):
            X = X.to_numpy()
            y = y.to_numpy()
        #x_train
        # if not torch.is_tensor(X):
        #     self.X = torch.from_numpy(X)
        # #y_train
        # if not torch.is_tensor(y):
        #     self.y = torch.from_numpy(y)
            
        #To convert boolean to int, do .long()
        self.X = torch.from_numpy(X)
        self.y = torch.from_numpy(y).long()
    def get_trainloader(dataset):
        return torch.utils.data.DataLoader(dataset, batch_size=10, shuffle=True, num_workers=1)
    def get_testloader(dataset):
        return torch.utils.data.DataLoader(dataset, batch_size=10, shuffle=True, num_workers=1)
    #to get lenght, for enumerator use
    def __len__(self):
        return len(self.X)
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

In [175]:
#Another attempt https://github.com/daenuprobst/theia/blob/main/src/theia/ml/mlp_classifier.py 
class theiaMLP(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(theiaMLP, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.fc1 = nn.Linear(self.input_size, self.hidden_size)
        self.tanh = nn.Tanh()
        self.fc2 = nn.Linear(self.hidden_size, self.output_size)

    def forward(self, x):
        hidden = self.fc1(x)
        tanh = self.tanh(hidden)
        output = self.fc2(tanh)
        return output
    

#initialize dataloader with random sampling of size 10 
dataset = dataLoader(X, y)
trainloader = torch.utils.data.DataLoader(dataset, batch_size=10, num_workers=0, shuffle = True)
testloader = torch.utils.data.DataLoader(dataset, batch_size=10, num_workers=0, shuffle=True)

#mlp init
mlp = theiaMLP(init_features, 1000, 1)
#set loss function and gradient descet optimizer
loss_function = nn.L1Loss()
optimizer = torch.optim.Adagrad(mlp.parameters(), lr=1e-4)

#train for this many epochs
for epoch in range(0,10):
    print(f'Starting Epoch {epoch+1}')

    current_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, targets = data
        inputs, targets = inputs.float(), targets.float()
        targets = targets.reshape((targets.shape[0], 1))
        optimizer.zero_grad()

        outputs = mlp(inputs)

        loss = loss_function(outputs, targets)

        loss.backward()

        optimizer.step()

        current_loss += loss.item()
        if i%10 == 0:
            print(f'Loss after mini-batch %5d: %.3f'%(i+1, current_loss/500))
            current_loss = 0.0
    
    print(f'Epoch {epoch+1} done')


test_data = torch.from_numpy(testX.to_numpy()).float()
test_targets = torch.from_numpy(testY.to_numpy()).float()

from sklearn.metrics import mean_squared_error, r2_score
with torch.no_grad():
    outputs = mlp(test_data)
    predicted_labels = outputs.squeeze().tolist()

predicted_labels = np.array(predicted_labels)
test_targets = np.array(test_targets)

mse = mean_squared_error(test_targets, predicted_labels)
r2 = r2_score(test_targets, predicted_labels)
print("Mean Squared Error:", mse)
print("R2 Score:", r2)
#Results
# Mean Squared Error: 0.11795077403554882
# R2 Score: 0.46766500127569677

Starting Epoch 1
Loss after mini-batch     1: 0.001
Loss after mini-batch    11: 0.007
Loss after mini-batch    21: 0.005
Loss after mini-batch    31: 0.006
Loss after mini-batch    41: 0.005
Loss after mini-batch    51: 0.006
Loss after mini-batch    61: 0.006
Loss after mini-batch    71: 0.006
Loss after mini-batch    81: 0.004
Loss after mini-batch    91: 0.005
Loss after mini-batch   101: 0.006
Loss after mini-batch   111: 0.006
Loss after mini-batch   121: 0.004
Loss after mini-batch   131: 0.005
Loss after mini-batch   141: 0.005
Loss after mini-batch   151: 0.005
Loss after mini-batch   161: 0.005
Loss after mini-batch   171: 0.006
Loss after mini-batch   181: 0.005
Loss after mini-batch   191: 0.004
Loss after mini-batch   201: 0.005
Loss after mini-batch   211: 0.006
Loss after mini-batch   221: 0.005
Loss after mini-batch   231: 0.005
Loss after mini-batch   241: 0.005
Loss after mini-batch   251: 0.006
Loss after mini-batch   261: 0.005
Loss after mini-batch   271: 0.005
Los

In [178]:
predicted_labels

array([ 5.32963753e-01,  6.39964402e-01,  4.29306656e-01,  1.70888945e-01,
        1.79741696e-01, -1.13772014e-02,  5.57978213e-01,  2.99851090e-01,
        5.12628376e-01,  5.02125442e-01,  4.82025921e-01,  5.21338642e-01,
        4.15739626e-01,  5.68720460e-01,  4.12976325e-01,  4.97355219e-03,
       -1.91447213e-01,  5.15680239e-02,  5.63815534e-01,  5.75622976e-01,
        6.65774882e-01,  6.59091055e-01,  3.02221239e-01,  4.89406198e-01,
        3.14879537e-01,  6.06696963e-01,  4.26411510e-01,  6.26404464e-01,
        5.78241348e-01,  6.09599948e-01,  6.03339314e-01,  8.39068830e-01,
        5.46481133e-01,  1.40842333e-01,  1.82300851e-01,  3.34612355e-02,
        4.65541855e-02,  1.13234282e-01, -5.72907273e-03,  1.93075165e-02,
        6.69669211e-01,  3.18893284e-01,  6.07983589e-01,  8.52727413e-01,
        5.83053827e-01,  8.14729095e-01,  1.01978624e+00,  2.45583281e-01,
        1.22324832e-01,  5.56691408e-01, -6.38945494e-03,  4.88863498e-01,
        5.80283582e-01,  

In [ ]:
#USING SKLEARN
from sklearn.neural_network import MLPClassifier
MAHOMES_clf = MLPClassifier(learning_rate_init = 0.01, activation='relu', hidden_layer_sizes= (100,), alpha = 0.001 )

#initialize dataloader with random sampling of size 10 
dataset = dataLoader(X, y)
trainloader = torch.utils.data.DataLoader(dataset, batch_size=10, num_workers=0, shuffle = True)
testloader = torch.utils.data.DataLoader(dataset, batch_size=10, num_workers=0, shuffle=True)

#mlp init
mlp = theiaMLP(init_features, 256, 1)
#set loss function and gradient descet optimizer
loss_function = nn.L1Loss()
optimizer = torch.optim.Adagrad(mlp.parameters(), lr=1e-4)

#train for this many epochs
for epoch in range(0,10):
    print(f'Starting Epoch {epoch+1}')

    current_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, targets = data
        inputs, targets = inputs.float(), targets.float()
        targets = targets.reshape((targets.shape[0], 1))
        optimizer.zero_grad()

        outputs = mlp(inputs)

        loss = loss_function(outputs, targets)

        loss.backward()

        optimizer.step()

        current_loss += loss.item()
        if i%10 == 0:
            print(f'Loss after mini-batch %5d: %.3f'%(i+1, current_loss/500))
            current_loss = 0.0
    
    print(f'Epoch {epoch+1} done')


test_data = torch.from_numpy(testX.to_numpy()).float()
test_targets = torch.from_numpy(testY.to_numpy()).float()
print("Test data correct outputs look like this", test_targets)

from sklearn.metrics import mean_squared_error, r2_score
with torch.no_grad():
    outputs = mlp(test_data)
    predicted_labels = outputs.squeeze().tolist()

predicted_labels = np.array(predicted_labels)
test_targets = np.array(test_targets)

mse = mean_squared_error(test_targets, predicted_labels)
r2 = r2_score(test_targets, predicted_labels)
print("Mean Squared Error:", mse)
print("R2 Score:", r2)
#Results
# Mean Squared Error: 0.10884078202201086
# R2 Score: 0.508780183660541

In [42]:
#to set num samples variable for dataset
num_samples = len(X)

In [100]:
import torch.utils.data.sampler as sampler
#Will use subsetRandomSampler (which assumes a shuffle=trfue data loading argument)

In [139]:
#initialize dataloader with random sampling of size 10 
dataset = dataLoader(X, y)
trainloader = torch.utils.data.DataLoader(dataset, batch_size=10, num_workers=0, shuffle = True)
testloader = torch.utils.data.DataLoader(dataset, batch_size=10, num_workers=0, shuffle=True)

In [140]:
#mlp init
mlp = MLP()
#set loss function and gradient descet optimizer
loss_function = nn.L1Loss()
optimizer = torch.optim.Adagrad(mlp.parameters(), lr=1e-4)

In [46]:
#check that enumerate works
enumerate(trainloader, 0)

In [35]:
def check_result_metrics(alg, feat_set, prediction_df):
    mcc = matthews_corrcoef(prediction_df['actual'], prediction_df['bool_pred'])
    TN, FP, FN, TP = confusion_matrix(prediction_df['actual'], prediction_df['bool_pred']).ravel()

    TPR=(TP/(TP+FN))*100
    TNR=(TN/(TN+FP))*100
    acc=((TP+TN)/(TP+TN+FP+FN))*100
    Prec=(TP/(TP+FP))*100
    return(pd.DataFrame([[alg, feat_set, acc, mcc, TPR, TNR, Prec]],
        columns=['Algorithm', 'Feature Set', 'Accuracy', 'MCC', 'Recall', 'TrueNegRate', 'Precision']))

In [ ]:
#train for this many epochs
for epoch in range(0,10):
    print(f'Starting Epoch {epoch+1}')

    current_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, targets = data
        inputs, targets = inputs.float(), targets.float()
        targets = targets.reshape((targets.shape[0], 1))
        optimizer.zero_grad()

        outputs = mlp(inputs)

        loss = loss_function(outputs, targets)

        loss.backward()

        optimizer.step()

        current_loss += loss.item()
        if i%10 == 0:
            print(f'Loss after mini-batch %5d: %.3f'%(i+1, current_loss/500))
            current_loss = 0.0
    
    print(f'Epoch {epoch+1} done')

In [143]:
test_data = torch.from_numpy(testX.to_numpy()).float()
test_targets = torch.from_numpy(testY.to_numpy()).float()
print("Test data correct outputs look like this", test_targets)

Test data correct outputs look like this tensor([1., 1., 1., 1., 0., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0.,
        0., 0., 0., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 1., 1., 1., 1., 1.,
        1., 1., 1., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 1., 1., 0., 1., 1.,
        1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0.,
        1., 1., 0., 1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 0., 1., 1., 1., 1., 1., 1., 0., 1., 1., 0., 0., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,

In [144]:
#Run mlp model on test data
mlp.eval() 

MLP(
  (layers): Sequential(
    (0): Linear(in_features=181, out_features=181, bias=True)
    (1): Sigmoid()
    (2): Linear(in_features=181, out_features=90, bias=True)
    (3): Sigmoid()
    (4): Linear(in_features=90, out_features=1, bias=True)
    (5): Sigmoid()
  )
)

In [145]:
predicted_labels

array([0.21674117, 0.22062142, 0.21141651, 0.21115166, 0.19262204,
       0.19279601, 0.22205086, 0.20826857, 0.21853697, 0.2159798 ,
       0.21711464, 0.21479124, 0.20975791, 0.21770474, 0.20866387,
       0.1990616 , 0.19156881, 0.1972757 , 0.22334646, 0.21868733,
       0.2250876 , 0.22253193, 0.21718466, 0.22030586, 0.21602862,
       0.21790211, 0.20592381, 0.2091603 , 0.21432683, 0.21131021,
       0.21055853, 0.21681574, 0.21286863, 0.19677123, 0.19910997,
       0.19341157, 0.20112966, 0.20306885, 0.19286221, 0.19625641,
       0.22222982, 0.20823163, 0.21841706, 0.22265032, 0.21728173,
       0.23444371, 0.23395725, 0.20475917, 0.20191696, 0.20894116,
       0.2050696 , 0.21854565, 0.21890941, 0.21993507, 0.20849989,
       0.22687183, 0.22544433, 0.23085712, 0.22505273, 0.22897832,
       0.23596846, 0.2183799 , 0.22118898, 0.21605703, 0.21403074,
       0.2239885 , 0.22787106, 0.20763335, 0.21379885, 0.20781212,
       0.21181458, 0.21015316, 0.22662792, 0.22248079, 0.21807

In [146]:
from sklearn.metrics import mean_squared_error, r2_score
with torch.no_grad():
    outputs = mlp(test_data)
    predicted_labels = outputs.squeeze().tolist()

predicted_labels = np.array(predicted_labels)
test_targets = np.array(test_targets)

mse = mean_squared_error(test_targets, predicted_labels)
r2 = r2_score(test_targets, predicted_labels)
print("Mean Squared Error:", mse)
print("R2 Score:", r2)
# Mean Squared Error: 0.18271737790066617
# R2 Score: 0.17536060337896386
# So absolute crap, but it's a start

Mean Squared Error: 0.23740676504140348
R2 Score: -0.07146333713221087
